In [15]:
from subprocess import Popen, PIPE
from sys import stdout, stderr
import os
import time
import dill as pickle
import json
import numpy as np
from utils import *

fw = open("tmpout", "wb")
fr = open("tmpout", "r")
p = Popen(["/Users/tianlins/Library/Android/sdk/platform-tools/adb", "shell"], stdin = PIPE, stdout = fw, stderr = fw, bufsize = 1, )



In [17]:
def block_on_result():
    while True:
        try:
            os.remove('result.txt')
        except:
            pass
        os.system('adb pull /sdcard/blas/result.txt ./')
        stdout.flush()
        if os.path.exists('result.txt'):
            with open('result.txt', 'r') as f:
                return f.read()
        time.sleep(1)
        
def test(align, **args):
    sa = args['sa']
    sb = args['sb']
    sc = args['sc']
    args['sa'] = int(np.ceil(args['sa'] / float(align)) * align)
    args['sb'] = int(np.ceil(args['sb'] / float(align)) * align)
    args['sc'] = int(np.ceil(args['sc'] / float(align)) * align)
    os.system('adb shell rm /sdcard/blas/result.txt')
    arg_str = ' '.join(['--' + key + " " + str(val) for (key, val) in args.items()]);
    p.stdin.write('''
    su
    cd /sdcard/blas
    export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:./
    ls
    ./gemm %s --output "result.txt"
    ''' % arg_str)
    result = block_on_result()
    try:
        result = json.loads(result)
    except ValueError:
        print 'json decoding error: ', result
    
    result['params']['sa'] = sa
    result['params']['sb'] = sb
    result['params']['sc'] = sc
    return result

In [16]:
test(cl_program='noblock-v8', arch='gpu', sa=96, sb=363, sc=3025, iterations=30)

json decoding error:  {
	"params": {
		"arithmetic": "float",
		"sa": 96,
		"sb": 352,
		"sc": 3008,
		"cl_program": "noblock-v8",
		"arch": "gpu"
	},
	"energy": nan,
	"time": 0.0542249,
	"gflops": 3.7828,
	"time_std": 0.00517215,
	"gflops_std": 0.0872941
}



'{\n\t"params": {\n\t\t"arithmetic": "float",\n\t\t"sa": 96,\n\t\t"sb": 352,\n\t\t"sc": 3008,\n\t\t"cl_program": "noblock-v8",\n\t\t"arch": "gpu"\n\t},\n\t"energy": nan,\n\t"time": 0.0542249,\n\t"gflops": 3.7828,\n\t"time_std": 0.00517215,\n\t"gflops_std": 0.0872941\n}\n'

In [12]:

test(cl_program='none', arch='viennacl', sa=96, sb=363, sc=3025, iterations=30)

json decoding error:  {
	"params": {
		"arithmetic": "float",
		"sa": 96,
		"sb": 352,
		"sc": 3008,
		"arch": "viennacl"
	},
	"energy": nan,
	"time": 0.16436,
	"gflops": 1.30608,
	"time_std": 0.0678306,
	"gflops_std": 0.00466228
}



'{\n\t"params": {\n\t\t"arithmetic": "float",\n\t\t"sa": 96,\n\t\t"sb": 352,\n\t\t"sc": 3008,\n\t\t"arch": "viennacl"\n\t},\n\t"energy": nan,\n\t"time": 0.16436,\n\t"gflops": 1.30608,\n\t"time_std": 0.0678306,\n\t"gflops_std": 0.00466228\n}\n'

In [18]:
global_results = load()

In [24]:
# conv layers.
s_list = [#(96, 363, 3025), 
          #(128, 1200, 729), 
          #(384, 2304, 169), 
          #(192, 1728, 169), 
          #(128, 1728, 169), 
            
            (384, 2304, 169)
         ]
num_iter = 50
# biases.
# s_list = [(96, 1, 3025), 
#           (256, 1, 729),
#           (384, 1, 169), 
#           (384, 1, 169),
#           (256, 1, 169)]
    

for s in s_list:
    (sa, sb, sc) = s
    print 'size', s, 'gpu1'
    result = test(align=16, cl_program='noblock-v8', arch='gpu', sa=sa, sb=sb, sc=sc, iterations=num_iter)
    print '\tgflops = ', result['gflops'], 'energy = ', result['energy']
    save(global_results, result)
    
    print 'size', s, 'gpu2'
    result = test(align=32, cl_program='blocking-2-v4', arch='gpu', sa=sa, sb=sb, sc=sc, iterations=num_iter)
    print '\tgflops = ', result['gflops'], 'energy = ', result['energy']
    save(global_results, result)
    
    print 'size', s, 'gpu3'
    result = test(align=64, cl_program='blocking-4-v4', arch='gpu', sa=sa, sb=sb, sc=sc, iterations=num_iter)
    print '\tgflops = ', result['gflops'], 'energy = ', result['energy']
    save(global_results, result)
    
    print 'size', s, 'cpu'
    result = test(align=32, cl_program='none', arch='cpu', sa=sa, sb=sb, sc=sc, iterations=num_iter)
    print '\tgflops = ', result['gflops'], 'energy = ', result['energy']
    save(global_results, result)
    
    print 'size', s, 'viennacl'
    result = test(align=16, cl_program='none', arch='viennacl', sa=sa, sb=sb, sc=sc, iterations=num_iter)
    print '\tgflops = ', result['gflops'], 'energy = ', result['energy']
    save(global_results, result)

size (384, 2304, 169) gpu1
	gflops =  4.39295 energy =  0.0867914
(384, 2304, 169, u'gpu', u'noblock-v8', u'float')
size (384, 2304, 169) gpu2
	gflops =  6.99622 energy =  0.0551246
(384, 2304, 169, u'gpu', u'blocking-2-v4', u'float')
size (384, 2304, 169) gpu3
	gflops =  4.60909 energy =  0.0922332
(384, 2304, 169, u'gpu', u'blocking-4-v4', u'float')
size (384, 2304, 169) cpu
	gflops =  18.2901 energy =  0.132648
(384, 2304, 169, u'cpu', 'none', u'float')
size (384, 2304, 169) viennacl
	gflops =  1.34021 energy =  0.0935604
(384, 2304, 169, u'viennacl', 'none', u'float')


In [63]:
print 'size', s, 'viennacl'
result = test(align=16, cl_program='none', arch='viennacl', sa=sa, sb=sb, sc=sc, iterations=num_iter)
print '\tgflops = ', result['gflops'], 'energy = ', result['energy']
save(global_results, result)

size (384, 1, 169) viennacl
	gflops =  0.260832 energy =  0.142059
(384, 16, 176, u'viennacl', 'none', u'float')


In [41]:
print 'size', s, 'viennacl'
result = test(align=16, cl_program='none', arch='viennacl', sa=sa, sb=sb, sc=sc, iterations=num_iter)
print '\tgflops = ', result['gflops'], 'energy = ', result['energy']
save(global_results, result)

size (96, 363, 3025) viennacl
	gflops =  1.19583 energy =  0.0770155
(96, 352, 3024, u'viennacl', 'none', u'float')


In [34]:
result = test(align=1, cl_program='none', arch='cpu', sa=sa, sb=sb, sc=sc, iterations=num_iter)

KeyboardInterrupt: 

In [10]:
global_results = load()

In [11]:
from IPython.display import display, HTML

def plot(x_dict, name_dict, target, **kwargs):
    names = name_dict.keys()
    names = sorted(names)
    x_bar = [(96, 363, 3025), 
          (128, 1200, 729), 
          (384, 2304, 169), 
          (192, 1728, 169),
          (128, 1728, 169)
         ]

    xs = []
    ys = []
    for name in names:
        x = []
        y = []
        for xi in x_bar:
            (sa, sb, sc) = xi
            yi = global_results[xi + name_dict[name]][target]
            y.append(yi)
            x.append(x_dict[xi])
        xs.append(x)
        ys.append(y)
    display(HTML(plot_bar(xs, ys, names, **kwargs)))


In [20]:
x_dict = {
    (96, 363, 3025): 'conv1',
    (96, 1, 3025): 'conv1-bias',
    (128, 1200, 729): 'conv2',
          (256, 1, 729): 'conv2-bias',
          (384, 2304, 169): 'conv3',
          (384, 1, 169): 'con3-bias',
          (192, 1728, 169): 'conv4',
          (384, 1, 169): 'conv4-bias',
          (128, 1728, 169): 'conv5', 
          (256, 1, 169): 'conv5-bias'
         }
plot(x_dict, {
        'gpu-blocking-2x2-vload4': (u'gpu', u'blocking-2-v4', 'float'),
        'gpu-blocking-4x4-vload4': (u'gpu', u'blocking-4-v4', 'float'),
        'gpu-no-blocking-vload8': (u'gpu', u'noblock-v8', 'float'),
        'cpu': (u'cpu', u'none', 'float'),
        'gpu-viennacl': (u'viennacl', 'none', 'float')
    }, 
     'time', xlabel='layer name', ylabel='time (ms)', 
     title='Processing time by convolution layer (AlexNet)')

<IPython.core.display.HTML object>

In [21]:
plot(x_dict, {
        'gpu-blocking-2x2-vload4': (u'gpu', u'blocking-2-v4', 'float'),
        'gpu-blocking-4x4-vload4': (u'gpu', u'blocking-4-v4', 'float'),
        'gpu-no-blocking-vload8': (u'gpu', u'noblock-v8', 'float'),
        'cpu': (u'cpu', u'none', 'float'),
        'gpu-viennacl': (u'viennacl', 'none', 'float')
    }, 
     'energy', xlabel='layer name', ylabel='power (mW)', 
     title='Energy consumption by convolution layer (AlexNet)')

<IPython.core.display.HTML object>

In [74]:
global_results

{(16, 16, 16, u'gpu', u'noblock-v8', u'float'): {u'energy': 0.193964,
  u'gflops': 0.00377686,
  u'gflops_std': 1.19882e-06,
  u'params': {u'arch': u'gpu',
   u'arithmetic': u'float',
   u'cl_program': u'noblock-v8',
   u'sa': 16,
   u'sb': 16,
   u'sc': 16},
  u'time': 0.00230433,
  u'time_std': 0.000713179},
 (32, 32, 32, u'gpu', u'blocking-2-v4', u'float'): {u'energy': 0.222426,
  u'gflops': 0.0298057,
  u'gflops_std': 1.54979e-05,
  u'params': {u'arch': u'gpu',
   u'arithmetic': u'float',
   u'cl_program': u'blocking-2-v4',
   u'sa': 32,
   u'sb': 32,
   u'sc': 32},
  u'time': 0.00231001,
  u'time_std': 0.000696565},
 (64, 64, 64, u'cpu', 'none'): {u'energy': 0.711669,
  u'gflops': 4.99246,
  u'gflops_std': 0.00513827,
  u'params': {u'arch': u'cpu', u'sa': 64, u'sb': 64, u'sc': 64},
  u'time': 0.000106431,
  u'time_std': 1.27757e-05},
 (64, 64, 64, u'gpu', u'blocking-2-v4'): {u'energy': 0.400894,
  u'gflops': 0.20074,
  u'gflops_std': 7.21521e-05,
  u'params': {u'arch': u'gpu',
   